In [1]:
import os

import geopandas as gpd
import nivapy3 as nivapy
import pandas as pd
import teotil3 as teo
import utils

In [2]:
eng = nivapy.da.connect_postgis()

Connection successful.


# OsloMod wastewater scenarios summary

Saving the OsloMod wastewater scenarios to Excel for comparison with previous Martini inputs.

In [3]:
st_yr, end_yr = 2017, 2019
vassom_list = range(1, 18)
oslomod_scen_dir = r"/home/jovyan/shared/common/oslofjord_modelling/phase3_scenarios"
scen_list = ["Baseline", "Scenario_A", "Scenario_B"]

In [4]:
# Get all WWTP site IDs within Oslofjord
df_list = []
for year in range(st_yr, end_yr + 1):
    site_df = teo.io.get_raw_annual_point_data(
        eng,
        year,
        "large wastewater",
        par_list=[
            "totn_kg",
            "din_kg",
            "ton_kg",
            "totp_kg",
            "tdp_kg",
            "tpp_kg",
            "toc_kg",
            "ss_kg",
        ],
    )[["site_id", "regine", "year"]]

    # Filter to vassoms of interest
    site_df["vassom"] = (
        site_df["regine"].str.split(".", n=1, expand=True)[0].astype(int)
    )
    site_df = (
        site_df.query("vassom in @vassom_list")
        .reset_index(drop=True)
        .rename(columns={"site_id": "anlegg_nr"})
    )
    df_list.append(site_df)
site_df = pd.concat(df_list, axis="rows")[["anlegg_nr", "regine"]].drop_duplicates()
assert site_df["anlegg_nr"].is_unique
site_id_list = site_df["anlegg_nr"].unique().tolist()
site_df.head()

,anlegg_nr,regine
0,0301.0979.01,006.21
1,0301.0980.01,006.A3
2,0301.1497.01,007.A0
3,3101.0051.01,001.1A2B
4,3101.0052.01,001.2220


In [5]:
# Get scenario data
df_list = []
for year in range(st_yr, end_yr + 1):
    for scen in scen_list:
        xl_path = os.path.join(
            oslomod_scen_dir,
            "teotil3_input_files",
            scen,
            str(year),
            f"large_wastewater_{year}_raw.xlsx",
        )
        df = pd.read_excel(xl_path).query("anlegg_nr in @site_id_list")
        df["scenario"] = scen
        df_list.append(df)
df = pd.concat(df_list, axis="rows")
df = pd.merge(df, site_df, how="left", on="anlegg_nr")

# Join Phil's Martini data
xl_path = r"../data/Flux_Martini_Rivers_sewage_with_mapping_and_fractions.xlsx"
names_dict = {
    "ANLEGGSNR": "kilderefnr",
    "Navn": "martini_name",
    "River_or_internal": "martini_river_or_internal",
}
mart_df = pd.read_excel(xl_path, sheet_name="RA_2018").rename(columns=names_dict)[
    names_dict.values()
]

# VEAS appears twice in Phil's data with the same kilderefnr.
# My data is just for the outlet, not the overflow
mart_df = mart_df.query("martini_name != 'VEAS-overflow'")

# Merge
df = pd.merge(df, mart_df, how="left", on="kilderefnr")
st_cols = [
    "scenario",
    "anlegg_nr",
    "kilderefnr",
    "anlegg_name",
    "martini_name",
    "martini_river_or_internal",
    "regine",
    "year",
]
cols = [col for col in df.columns if col not in st_cols]
df = (
    df[st_cols + cols]
    .sort_values(["scenario", "anlegg_nr", "year"])
    .reset_index(drop=True)
)

# Round
cols = [col for col in df.columns if col.endswith("_tonnes")]
df[cols] = df[cols].round(3)

xl_path = r"../data/wwtp_scenarios_summary.xlsx"
df.to_excel(xl_path, index=False)

df.head()

,scenario,anlegg_nr,kilderefnr,anlegg_name,martini_name,martini_river_or_internal,regine,year,activity,kommune,...,bof5_in_tonnes,bof5_out_tonnes,kof_in_tonnes,kof_out_tonnes,ss_in_tonnes,ss_out_tonnes,totn_in_tonnes,totn_out_tonnes,totp_in_tonnes,totp_out_tonnes
0,Baseline,0301.0979.01,0301AL01,Bekkelaget renseanlegg med tilførselstuneller ...,Bekkelaget RA,I,006.21,2017,Avløpsnett og -rensing,Oslo,...,8063.605,260.256,19753.086,1468.092,7515.890,526.112,1484.844,428.126,171.453,17.192
1,Baseline,0301.0979.01,0301AL01,Bekkelaget renseanlegg med tilførselstuneller ...,Bekkelaget RA,I,006.21,2018,Avløpsnett og -rensing,Oslo,...,7462.153,266.345,17150.180,1464.857,7579.970,530.598,1505.006,390.966,178.464,15.728
2,Baseline,0301.0979.01,0301AL01,Bekkelaget renseanlegg med tilførselstuneller ...,Bekkelaget RA,I,006.21,2019,Avløpsnett og -rensing,Oslo,...,6974.607,416.871,18235.991,2017.199,7579.970,530.598,1423.836,484.368,171.257,22.774
3,Baseline,0301.0980.01,0301AL27,Grefsenkollen renseanlegg,NaN,NaN,006.A3,2017,Privat avløpsanlegg,Oslo,...,0.066,0.016,0.131,0.033,0.077,0.009,0.013,0.010,0.002,0.000
4,Baseline,0301.0980.01,0301AL27,Grefsenkollen renseanlegg,NaN,NaN,006.A3,2018,Privat avløpsanlegg,Oslo,...,0.066,0.016,0.131,0.033,0.077,0.009,0.013,0.010,0.002,0.000
